<a href="https://colab.research.google.com/github/xquared-design/DCGan_calligraphy/blob/main/dcgan_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install libs first because you will need to 

In [ ]:
!pip install cupy-cuda100==7.4.0 tensorflowjs==1.2.6
!pip install --upgrade six

get the ml5 versoin of training dcgan

In [ ]:
!git clone https://github.com/ml5js/training-dcgan
%cd ./training-dcgan

Cloning into 'training-dcgan'...
remote: Enumerating objects: 100, done.
remote: Total 100 (delta 0), reused 0 (delta 0), pack-reused 100
Receiving objects: 100% (100/100), 151.41 KiB | 6.06 MiB/s, done.
Resolving deltas: 100% (39/39), done.


In [ ]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0   130k      0 --:--:-- --:--:-- --:--:--  130k
OK
61 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.37.0).
0 upgraded, 0 newly installed, 0 to remove and 61 not upgraded.


Mapping Google cloud buckey holding the original image to the images folder

In [ ]:
#@title Google Cloud bucket name
!rm -r ./images && mkdir images
gs_name = 'calligraphy-dcgan-training-data' #@param {type:"string"}
!gcsfuse gs_name images/

2021/11/14 06:13:35.093847 Start gcsfuse/0.37.0 (Go version go1.17.2) for app "" using mount point: /content/training-dcgan/training-dcgan/images
2021/11/14 06:13:35.100576 Opening GCS connection...
2021/11/14 06:13:35.275310 Mounting file system "calligraphy-dcgan-training-data"...
2021/11/14 06:13:35.276782 File system has been successfully mounted.


In [ ]:
# Next we convert png images to jpeg (because the ml5js/training-dcgan code
# doesn't like images with transparency).

!apt install imagemagick
!cd ./images && mogrify -format jpg *.gif && rm *.gif && mogrify -format jpg *.png && rm *.png

In [ ]:
!python datatool.py --size 256

In [ ]:
# keep batch number small to avoid running our of memory
!python chainer_dcgan.py --arch resnet256 --image_size 256 --npz_path /content/drive/MyDrive/dataset.npz --batch_size 32 --max_iter 10000 --out /content/drive/MyDrive/results

/usr/local/lib/python3.7/dist-packages/chainer/_environment_check.py:75: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy-cuda111) version 9.4.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy-cuda111>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  requirement=requirement, help=help))
/usr/local/lib/python3.7/dist-packages/chainer/_environment_check.py:75: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy-cuda100) version 9.6.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy-cuda100>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/l

In [ ]:
%cp --verbose result/*.npz /content/drive/MyDrive/results
%cp result/preview*/*.jpg

In [ ]:

!python dcgan_chainer_to_keras.py --arch resnet256 --chainer_model_path /content/drive/MyDrive/results/SmoothedGenerator_10000.npz


Tensorflow.js model path:  /content/drive/MyDrive/results/SmoothedGenerator_10000_tfjs
weight
--------------------------------------------------------------------------------
dense/l/W (16384, 128)
dense/l/b (16384,)
finals/0/N ()
finals/0/avg_mean (32,)
finals/0/avg_var (32,)
finals/0/beta (32,)
finals/0/gamma (32,)
finals/2/W (3, 32, 3, 3)
finals/2/b (3,)
resblockups/0/bn0/N ()
resblockups/0/bn0/avg_mean (1024,)
resblockups/0/bn0/avg_var (1024,)
resblockups/0/bn0/beta (1024,)
resblockups/0/bn0/gamma (1024,)
resblockups/0/bn1/N ()
resblockups/0/bn1/avg_mean (1024,)
resblockups/0/bn1/avg_var (1024,)
resblockups/0/bn1/beta (1024,)
resblockups/0/bn1/gamma (1024,)
resblockups/0/c0/W (1024, 1024, 3, 3)
resblockups/0/c0/b (1024,)
resblockups/0/c1/W (1024, 1024, 3, 3)
resblockups/0/c1/b (1024,)
resblockups/0/cs/W (1024, 1024, 3, 3)
resblockups/0/cs/b (1024,)
resblockups/1/bn0/N ()
resblockups/1/bn0/avg_mean (1024,)
resblockups/1/bn0/avg_var (1024,)
resblockups/1/bn0/beta (1024,)
resblockups/

In [ ]:
%cp -r -v SmoothedGenerator_50000_tfjs /content/drive/MyDrive/results

cp: cannot stat 'SmoothedGenerator_50000_tfjs': No such file or directory
